1. 导入模块将wav无损音频分离到audio文件夹中

In [2]:
import os
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tqdm import tqdm, trange
import time
from moviepy.editor import *
video_path="./video"
audio_path="./audio"

In [3]:
video_list=os.listdir(video_path)
for file in tqdm((video_list),desc='processing'):
    if('.mp4' in file):
        video = VideoFileClip(video_path+"/"+file)
        audio = video.audio
        #print(file.replace(".mp4", ".wav"))
        (audio.write_audiofile(audio_path+'/'+file.replace(".mp4", ".wav")))

processing:   5%|▌         | 1/20 [00:00<00:08,  2.29it/s]          

MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch06.wav
MoviePy - Done.


processing:  10%|█         | 2/20 [00:00<00:07,  2.38it/s]          

MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch06.wav
MoviePy - Done.


processing:  15%|█▌        | 3/20 [00:01<00:07,  2.33it/s]          

...

MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch09.wav
MoviePy - Done.


processing:  95%|█████████▌| 19/20 [00:08<00:00,  2.06it/s]         

MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch08.wav
MoviePy - Done.


processing: 100%|██████████| 20/20 [00:09<00:00,  2.17it/s]         

MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch08.wav
MoviePy - Done.


2.使用librosa进行音频处理
2.1 120帧的video和60帧的audio

In [4]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torch
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [25]:
data=list()
for audiofile in tqdm(os.listdir(audio_path),desc="processing audio"):
    envelop = audiofile
    start=0
    duration = 4
    fps=60
    stop = start + duration
    y,sr=librosa.load(audio_path+"/"+audiofile,sr=None,offset = start, duration = duration)#sr是采样率,None指原生采样率

    audio_duration = librosa.get_duration(y,sr)
    '''
    分帧就是将原始语音信号分成大小固定的N段语音信号，这里每一段语音信号都被称为一帧，
    帧长一般取10到30ms。分帧一般采用交叠分段的方法，是为了使帧与帧之间平滑过渡
    保持其连续性。前一帧和后一帧的交叠部分称为帧移，帧移与帧长的比值一般取为0-1/2。
    '''
    y,sr
    audio_dist = y
    frame_shift = int(sr/(fps+4))#帧移，保证60个都有正常数据 4代表着framelength和shift之间的比值
    frame_length = frame_shift*4#帧长
    #print(audio_dist[0].dtype) float 32
    #audio_dist=np.append(np.array(audio_dist),np.zeros(frame_length-frame_shift))#这里补0
    hop_length =int(frame_shift) #frame_shift * sr窗口每部滑动距离（跳长）
    win_length = int(frame_length)#窗口样本
    hop_length
    '''
    plt.figure()
    librosa.display.waveplot(audio_dist,sr)
    #librosa.display.waveplot(audio_dist,sr)
    '''
    mfccs=np.array([])
    chroma=np.array([])
    peak=np.array([])
    #tempo=np.array([])
    beats=np.array([])
    for i in range(fps*duration):
        '''
        mfcc返回 (n_mfcc,a)的向量，a=ceil(autioframelength / 帧移)
        '''
        print(audio_dist[i*frame_shift:i*frame_shift+frame_length])
        mfccs_tmp=librosa.feature.mfcc(y=audio_dist[i*frame_shift:i*frame_shift+frame_length],sr=sr,n_mfcc=20,hop_length=frame_length)
        print(mfccs_tmp.shape)
        chroma_tmp=librosa.feature.chroma_stft(audio_dist[i*frame_shift:i*frame_shift+frame_length],sr=sr,hop_length=hop_length,win_length=win_length,n_chroma=12)

        oneset_env_tmp = librosa.onset.onset_strength(y=audio_dist[i*frame_shift:i*frame_shift+frame_length],sr=sr,hop_length=hop_length,aggregate=np.median)
        peak_tmp = librosa.util.peak_pick(oneset_env_tmp,1,1,1,1,0.8,5)#公式来决定的，具体参数最后两个比较重要，后面需要调整
        tempo,beats_tmp = librosa.beat.beat_track(y=audio_dist[i*frame_shift:i*frame_shift+frame_length],sr=sr,hop_length=hop_length)
        print(mfccs_tmp.shape)
        break
    '''
    value=np.array(peak)
    label_encoder=LabelEncoder()
    integer_encoded = label_encoder.fit_transform(value)
    print(integer_encoded)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print(onehot_encoded)
    '''


    #print("tempo",tempo, "\nbeat_frames",beats)
    #tmp=envelop+mfccs+chroma+peak+beats
    envelop=np.asarray(envelop)
    envelop=np.asarray([[[envelop]]])
    mfccs=np.asarray([mfccs])
    chroma=np.asarray([chroma])
    peak=np.asarray([[peak]])
    beats=np.asarray([[beats]])
    print(envelop.shape)
    print(mfccs.shape)
    print(chroma.shape)
    print(peak.shape)
    print(beats.shape)


    tmp=np.concatenate((envelop,mfccs),axis=1)
    print(tmp.shape)

    '''
    # Display click waveform next to the spectrogram
    plt.figure()
    S = librosa.feature.melspectrogram(y=audio_dist, sr=sr)
    ax = plt.subplot(2,1,2)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max),
                             x_axis='time', y_axis='mel')
    plt.subplot(2,1,1, sharex=ax)
    librosa.display.waveplot(beats, sr=sr, label='Beat clicks')
    plt.legend()
    plt.xlim(15, 30)
    plt.tight_layout()
    '''
    break



processing audio:   0%|          | 0/20 [00:00<?, ?it/s]

[-3.05175781e-05  0.00000000e+00  0.00000000e+00 ... -4.54467773e-01
 -4.80148315e-01 -3.85620117e-01]
(20, 2)


ParameterError: Target size (2048) must be at least input size (2940)